# Enabling Deep Learning in Gravitational Wave Physics With Inference-as-a-Service
## Alec Gunny

## Deep Learning in Gravitational Wave Astronomy
<img src="images/gw-dl-use-cases.png" height="auto" width="980px" style="top:10px;"></img>

## Inference Time Challenges
<div class="float200 padded">
    <img src="images/hardware-logos.png" width="300px" height="auto" class="right unpadded"/>
    <p class="left">Access to and familiarity with accelerated hardware</p>
</div>

<div>
    <img src="images/framework-logos.png" width="300px" height="auto" class="left padded" />
    <p class="right"> Managing, leveraging, and translating across deep learning software stacks
</div>

<div></div>
<div></div>
<div class="float100">
    <img src="images/distributing-nns.png" width="250px" height="auto" class="right unpadded"/>
    <p class="left">Distributing updated models to dependent users and applications</p>
</div>

## A simple PyTorch example
- Naive inference pipeline for a model trained using PyTorch
- Pipeline itself loads the model and data onto the GPU and executes inference

In [1]:
import time
from concurrent.futures import ThreadPoolExecutor
from queue import Empty, Queue

import numpy as np
import torch

import utils

Instantiate a simple model, "train" it, then save it for inference:

In [6]:
class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super().__init__()

        self.layers = torch.nn.ModuleList()
        for size in hidden_sizes:
            self.layers.append(torch.nn.Linear(input_size, size))
            self.layers.append(torch.nn.ReLU())
            input_size = size

        self.layers.append(torch.nn.Linear(input_size, 1))
        self.layers.append(torch.nn.Sigmoid())

    def forward(self, x: torch.tensor) -> torch.tensor:
        for layer in self.layers:
            x = layer(x)
        return x

INPUT_SIZE, HIDDEN_SIZES = 64, [256, 128, 64]
model = MLP(INPUT_SIZE, HIDDEN_SIZES).cuda(device=0)

# do_some_training(model) -> export the trained weights
torch.save(model.state_dict(), "model.pt")

At inference time, load in the model weights and use it to map inputs to outputs

In [7]:
new_model = MLP(INPUT_SIZE, HIDDEN_SIZES).cuda(0)
new_model.load_state_dict(torch.load("model.pt"))

# create an array on the CPU
x = np.random.randn(INPUT_SIZE).astype("float32")
with torch.no_grad():
    # move it on to the GPU
    x = torch.from_numpy(x).cuda(0)

    # map it to outputs on the GPU
    y = model(x)

# move the outputs back on to the CPU
y.cpu().numpy()

array([0.51708245], dtype=float32)

- Generally interested in using the model for many thousands or millions of inferences
- Start with the simplest case: a dataset that can fit into memory at once

In [13]:
dataset = np.random.randn(10**5, 64).astype("float32")

@torch.no_grad()
def do_some_inference(model, dataset, batch_size=8):
    # move the data to the GPU in bulk
    gpu_dataset = torch.from_numpy(dataset).cuda(0)

    # iterate through it in batches and yield predictions
    dataset = torch.utils.data.TensorDataset(gpu_dataset)
    for [x] in torch.utils.data.DataLoader(dataset, batch_size=batch_size):
        y = model(x)
        yield y.cpu().numpy()

# run through the dataset and get a rough time estimate
%time outputs = [y for y in do_some_inference(new_model, dataset)]

CPU times: user 3.34 s, sys: 28 ms, total: 3.37 s
Wall time: 3.37 s


Ok, but how well are we utilizing the GPU?

In [14]:
with utils.get_progbar([0]) as progbar:
    task_id = progbar.add_task("[cyan]Inference", total=len(dataset))

    outputs = []
    for y in do_some_inference(model, dataset):
        outputs.append(y)
        progbar.update(task_id, advance=len(y))

output = np.concatenate(outputs, axis=0)

Output()

- Not very well! GPUs are expensive, can we improve things via parallel execution?
- First attempt: Naive (and sloppy) implementation using threading

In [15]:
q = Queue()
def task(dataset_chunk):
    model = utils.MLP(64, [256, 128, 64]).cuda(0)
    model.load_state_dict(torch.load("model.pt"))

    for y in do_some_inference(model, dataset_chunk):
        q.put(y)

with utils.get_progbar([0]) as progbar, ThreadPoolExecutor(4) as pool:
    task_id = progbar.add_task("Inference with 4 jobs", total=len(dataset))
    [pool.submit(task, x) for x in np.array_split(dataset, 4)]

    outputs = []
    while len(outputs) < len(dataset):
        y = q.get()
        progbar.update(task_id, advance=len(y))
        outputs.extend(y)

Output()

Not much help. What next?

After spending a few hours perusing the PyTorch documentation, we come up with the following basic functional implementation:

In [20]:
def parallel_inference(X, num_gpus, jobs_per_gpu, progbar):
    num_jobs = num_gpus * jobs_per_gpu
    task_id = progbar.add_task(
        f"[cyan]{num_gpus} GPUs/{num_jobs} jobs",
        total=len(X),
        start=False
    )

    # we need special queue and value objects
    # specific to process spawning
    smp = torch.multiprocessing.get_context("spawn")
    q = smp.Queue()
    starter = smp.Value("d", 0.0)

    # spawn a bunch of parallel jobs across all GPUs
    # have to host the `parallel_inference_task` in
    # a separate module for weird multiprocessing reasons
    procs = torch.multiprocessing.spawn(
        utils.parallel_inference_task,
        args=(jobs_per_gpu, X, q, starter, num_gpus),
        nprocs=num_jobs,
        join=False
    )

    # wait to synchronize until all models load
    # so that we can compare throughput better
    while starter.value < num_jobs:
        time.sleep(0.01)

    # increment the starter to kick off the jobs
    starter.value += 1
    progbar.start_task(task_id)

    # collect all the (unordered) inputs
    outputs = []
    while True:
        try:
            y = q.get_nowait()
            progbar.update(task_id, advance=len(y))
            outputs.append(y)
        except Empty:
            # if there's nothing in the queue and
            # all the jobs are dead, we're done
            if procs.join(0.01):
                break

    # concatenate the outputs and return
    return np.concatenate(outputs, axis=0)

Run this at a few different scales to see what we can achieve:

In [21]:
with utils.get_progbar([0, 1]) as progbar:
    y = parallel_inference(dataset, num_gpus=1, jobs_per_gpu=2, progbar=progbar)
    y = parallel_inference(dataset, num_gpus=1, jobs_per_gpu=4, progbar=progbar)
    y = parallel_inference(dataset, num_gpus=2, jobs_per_gpu=4, progbar=progbar)

Output()

It works pretty well! But:
- We got lucky: this wouldn't be possible in most other frameworks
- The code is complicated and required a lot of non-physics expertise to build
    - Non-trivial to reconstruct for new applications
- Extremely contrived example, breaks down in most real use cases